In [11]:

from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

In [12]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [13]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'outputDir'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
# USE_BUCKET = False #@param {type:"boolean"}
# BUCKET = 'BUCKET_NAME' #@param {type:"string"}

# if USE_BUCKET:
#   OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
#   from google.colab import auth
#   auth.authenticate_user()

# if DO_DELETE:
#   try:
#     tf.gfile.DeleteRecursively(OUTPUT_DIR)
#   except:
#     # Doesn't matter if the directory didn't exist
#     pass
# tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: outputDir *****


In [14]:
import pandas as pd
train = pd.read_csv('./insults_data/train.csv')
test = pd.read_csv('./insults_data/test.csv')
valid = pd.read_csv('./insults_data/valid.csv')

In [15]:
import numpy as np
train.loc[:,'Class'] = np.where(train.Class == 'Insult', 1, 0)
test.loc[:,'Class'] = np.where(test.Class == 'Insult', 1, 0)
valid.loc[:,'Class'] = np.where(valid.Class == 'Insult', 1, 0)

In [16]:
train.head()

,Comment,Class
0,"""You fuck your dad.""",1
1,"""i really don't understand your point.\xa0 It ...",0
2,"""A\\xc2\\xa0majority of Canadians can and has ...",0
3,"""listen if you dont wanna get married to a man...",0
4,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd...",0


In [17]:
test.head()

,Comment,Class
0,"""like this if you are a tribe fan""",0
1,"""you're idiot.......................""",1
2,"""I am a woman Babs, and the only ""war on women...",1
3,"""WOW & YOU BENEFITTED SO MANY WINS THIS YEAR F...",1
4,"""haha green me red you now loser whos winning ...",1


In [18]:
valid.head()

,Comment,Class
0,"""THE DRUDGE REPORT\\n\\n\\n\\nYou won't see th...",0
1,"""@ian21\xa0""Roger Clemens is the fucking man, ...",0
2,"""Agree with Alan you are an extremest idiot. ...",1
3,"""Really?\\n\\nI see Marc Lamont Hill on variou...",0
4,"""Really suck isn't the word, when many of our ...",0


In [19]:
DATA_COLUMN = 'Comment'
LABEL_COLUMN = 'Class'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

In [20]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [21]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [22]:
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

In [23]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 3947


INFO:tensorflow:Writing example 0 of 3947


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 1 (id = 1)


INFO:tensorflow:label: 1 (id = 1)


In [24]:
train_features[0].input_ids

[101,
 1000,
 2017,
 6616,
 2115,
 3611,
 1012,
 1000,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [25]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  input_ids = tf.cast(input_ids, tf.int32)
  input_mask = tf.cast(input_mask, tf.int32)
  segment_ids = tf.cast(segment_ids, tf.int32)

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [26]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [27]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [28]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [29]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [30]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': 'outputDir', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4d982e1a50>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'outputDir', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4d982e1a50>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [31]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [32]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Skipping training since max_steps has already saved.


INFO:tensorflow:Skipping training since max_steps has already saved.


Training took time  0:00:00.011419


In [33]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [34]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


...

INFO:tensorflow:Saving dict for global step 370: auc = 0.77140373, eval_accuracy = 0.7758389, f1_score = 0.73617685, false_negatives = 378.0, false_positives = 123.0, global_step = 370, loss = 0.8937263, precision = 0.850365, recall = 0.6490251, true_negatives = 1035.0, true_positives = 699.0


INFO:tensorflow:Saving dict for global step 370: auc = 0.77140373, eval_accuracy = 0.7758389, f1_score = 0.73617685, false_negatives = 378.0, false_positives = 123.0, global_step = 370, loss = 0.8937263, precision = 0.850365, recall = 0.6490251, true_negatives = 1035.0, true_positives = 699.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 370: outputDir/model.ckpt-370


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 370: outputDir/model.ckpt-370


{'auc': 0.77140373,
 'eval_accuracy': 0.7758389,
 'f1_score': 0.73617685,
 'false_negatives': 378.0,
 'false_positives': 123.0,
 'loss': 0.8937263,
 'precision': 0.850365,
 'recall': 0.6490251,
 'true_negatives': 1035.0,
 'true_positives': 699.0,
 'global_step': 370}

In [35]:
def getPrediction(in_sentences):
  labels = ["Positive", "Negative"]
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn, yield_single_examples=False)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [36]:
pred_sentences = [
  "That is fucking awesome"
]

In [37]:
predictions = getPrediction(pred_sentences)

INFO:tensorflow:Writing example 0 of 1


INFO:tensorflow:Writing example 0 of 1


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from outputDir/model.ckpt-370


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [38]:
predictions

[('That is fucking awesome',
  array([[-0.00562691, -5.183012  ]], dtype=float32),
  'Positive')]

In [ ]:
# your_feature_spec = {
#     # "some_feature": tf.FixedLenFeature([], dtype=tf.string, default_value=""),
#     "Comment": tf.VarLenFeature(dtype=tf.string),
# }

# def _serving_input_receiver_fn():
#     serialized_tf_example = tf.placeholder(dtype=tf.string, shape=None, 
#                                            name='input_example_tensor')
#     # key (e.g. 'examples') should be same with the inputKey when you 
#     # buid the request for prediction
#     receiver_tensors = {'examples': serialized_tf_example}
#     features = tf.parse_example(serialized_tf_example, your_feature_spec)
#     return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)

# # estimator.export_savedmodel(export_dir, _serving_input_receiver_fn)

In [ ]:
train_features[0]

In [ ]:
def serving_input_receiver_fn():
  feature_spec = {
      "input_ids" : tf.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
      "input_mask" : tf.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
      "segment_ids" : tf.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
      "label_ids" :  tf.FixedLenFeature([], tf.int64)
  }
  serialized_tf_example = tf.placeholder(dtype=tf.string,
                                         shape=None,
                                         name='input_example_tensor')
  print(serialized_tf_example.shape)
  receiver_tensors = {'example': serialized_tf_example}
  features = tf.parse_example(serialized_tf_example, feature_spec)
  return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)

export_path = './savedModel'
estimator._export_to_tpu = False  # this is important
estimator.export_saved_model(export_dir_base=export_path,serving_input_receiver_fn=serving_input_receiver_fn)

In [ ]:
"examples"=[{"input_ids":[b"\n\x12\n\x10\n\x08sentence\x12\x04\n\x02\n\x00"],"input_mask":[b"\n-\n+\n\x08sentence\x12\x1f\n\x1d\n\x1bThis API is a little tricky"],"segment_ids":[None],"label_ids":["label_1"]}]

In [ ]:
from tensorflow.contrib import predictor

predict_fn = predictor.from_saved_model("./savedModel/1580195293")

In [ ]:
!saved_model_cli show --all --dir ./savedModel/1580195293

In [ ]:
predict_fn({"example":'[{"input_ids":[b"sentence"],"input_mask":[b"This API is a little tricky"],"segment_ids":[None],"label_ids":["label_1"]}]'})

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.python.saved_model import tag_constants

flags = tf.flags

FLAGS = flags.FLAGS

flags.DEFINE_integer(
    "max_seq_length", 128,
    "The maximum total input sequence length after WordPiece tokenization. "
    "Sequences longer than this will be truncated, and sequences shorter "
    "than this will be padded.")

flags.DEFINE_string(
    "export_dir", './savedModel/1580195293',
    "The dir where the exported model has been written.")

flags.DEFINE_string(
    "predict_file", test_InputExamples,
    "The file of predict records")

graph = tf.Graph()
with graph.as_default():
    with tf.Session() as sess:
        tf.saved_model.loader.load(sess, [tag_constants.SERVING], FLAGS.export_dir)
        tensor_input_ids = graph.get_tensor_by_name('input_ids_1:0')
        tensor_input_mask = graph.get_tensor_by_name('input_mask_1:0')
        tensor_label_ids = graph.get_tensor_by_name('label_ids_1:0')
        tensor_segment_ids = graph.get_tensor_by_name('segment_ids_1:0')
        tensor_outputs = graph.get_tensor_by_name('loss/Softmax:0')
        record_iterator = tf.python_io.tf_record_iterator(path=FLAGS.predict_file)
        for string_record in record_iterator:
            example = tf.train.Example()
            example.ParseFromString(string_record)
            input_ids = example.features.feature['input_ids'].int64_list.value
            input_mask = example.features.feature['input_mask'].int64_list.value
            label_ids = example.features.feature['label_ids'].int64_list.value
            segment_ids = example.features.feature['segment_ids'].int64_list.value
            result = sess.run(tensor_outputs, feed_dict={
                tensor_input_ids: np.array(input_ids).reshape(-1, FLAGS.max_seq_length),
                tensor_input_mask: np.array(input_mask).reshape(-1, FLAGS.max_seq_length),
                tensor_label_ids: np.array(label_ids),
                tensor_segment_ids: np.array(segment_ids).reshape(-1, FLAGS.max_seq_length),
            })
            print(*(result[0]), sep='\t')

In [ ]:
# "input_ids" : tf.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
#       "input_mask" : tf.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
#       "segment_ids" : tf.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
#       "label_ids" :  tf.FixedLenFeature([], tf.int64)

examples = [
    tf.train.Example(
      features=tf.train.Features(
        feature={"input_ids": tf.train.Feature(
          int=tf.train.Int32(value=[1]*MAX_SEQ_LENGTH)),
          "input_mask": tf.train.Feature(
          int=tf.train.Int32(value=[1]*MAX_SEQ_LENGTH)),
          "segment_ids": tf.train.Feature(
          int=tf.train.Int32(value=[1]*MAX_SEQ_LENGTH)),
          "label_ids": tf.train.Feature(
          int=tf.train.Int32(value=[1]*MAX_SEQ_LENGTH))})),
  ]



predict_fn(examples)

# "input_ids" : tf.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
# "input_mask" : tf.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
# "segment_ids" : tf.FixedLenFeature([MAX_SEQ_LENGTH], tf.int64),
# "label_ids" :  tf.FixedLenFeature([], tf.int64)

In [ ]:
ls